In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
files = glob('threads/*.json')
len(files)

21583

In [3]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [5]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    for d in data:
        if d[-1] > 1:
            url = d[0]
            for i in range(2, d[-1] + 1, 1):
                urls.append(f'{url}page-{i}')

100%|███████████████████████████████████| 21583/21583 [00:18<00:00, 1156.57it/s]


In [11]:
urls = sorted(list(set(urls)))

In [12]:
len(urls)

343

In [13]:
urls

['https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-10',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-11',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-12',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-13',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-14',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-15',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-16',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-17',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-18',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-19',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-2',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-20',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-21',
 'https://b.cari.com.my/forum.php?mod=viewthread&tid=654327page-22',
 'https://b.cari.com.my/forum.php?m

In [14]:
def get_url(url):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    td = soup.find_all('td', {'class': 't_f'})
    td = [str(t) for t in td]
    return td

In [10]:
max_worker = 50
for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
    
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_url, url): url for url in urls[i: i + max_worker]}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

100%|█████████████████████████████████████████████| 7/7 [01:11<00:00, 10.27s/it]


In [17]:
files = glob('threads/*.json')
len(files)

21583

In [18]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d[1]:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|█████████████████████████████████████| 21583/21583 [12:46<00:00, 28.14it/s]


In [19]:
files = glob('pages/*.json')
len(files)

7

In [20]:
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|█████████████████████████████████████████████| 7/7 [00:18<00:00,  2.61s/it]


In [21]:
texts = list(set(texts))
len(texts)

4444727

In [22]:
with open('everything.jsonl', 'w') as fopen:
    for t in tqdm(texts):
        if len(t):
            fopen.write(f'{json.dumps(t)}\n')

100%|█████████████████████████████| 4444727/4444727 [00:06<00:00, 660471.89it/s]


In [24]:
# !zip -r threads.zip threads
# !zip -r pages.zip pages
# !zip -r forums.zip forums